In [40]:
import rope.base.project
from rope.refactor.extract import ExtractMethod
import re

In [48]:
APOSTROPHE_MARKER = "__APOSTROPHE__"
PERCENT_FORMAT_MARKER = "__PERCENT_FORMAT__"

def serialize(text: str):
    # Replace "'{var}'" with "__APOSTROPHE__{var}__APOSTROPHE__"
    text = re.sub(r"'{([^'}]*?)}'", f"{APOSTROPHE_MARKER}{{\\1}}{APOSTROPHE_MARKER}", text)
    # Replace "%s" with "__PERCENT_FORMAT__"
    text = re.sub(r"%\((.*?)\)s", f"{PERCENT_FORMAT_MARKER}{{\\1}}", text)
    return text

def deserialize(text: str):
    text = re.sub(f"{APOSTROPHE_MARKER}{{(.*?)}}{APOSTROPHE_MARKER}", "'{\\1}'", text)
    text = re.sub(f"{PERCENT_FORMAT_MARKER}{{(.*?)}}", "%(\\1)s", text)
    return text

myproject = rope.base.project.Project('src')

myresource = myproject.get_resource('test2.py')
contents = myresource.read()
serialized_contents = serialize(myresource.read())
myresource.write(serialized_contents)
extract_span = r'''for (git, ref), group_deps in grouped.items():
        if len(group_deps) == 1:
            typer.echo(f"Adding {git}@{ref}...")
        else:
            typer.echo(f"Adding {len(group_deps)} templates from {git}@{ref}")
        source_repo_path = update_repo(git, ref, REPO_DIR)

        for dep in group_deps:
            source_path = (
                source_repo_path / dep["subdirectory"]
                if dep["subdirectory"]
                else source_repo_path
            )
            pyproject_path = source_path / "pyproject.toml"
            if not pyproject_path.exists():
                typer.echo(f"Could not find {pyproject_path}")
                continue
            langserve_export = get_langserve_export(pyproject_path)

            # default path to package_name
            inner_api_path = dep["api_path"] or langserve_export["package_name"]

            destination_path = package_dir / inner_api_path
            if destination_path.exists():
                typer.echo(
                    f"Folder {str(inner_api_path)} already exists. " "Skipping...",
                )
                continue
            copy_repo(source_path, destination_path)
            typer.echo(f" - Downloaded {dep['subdirectory']} to {inner_api_path}")
            installed_destination_paths.append(destination_path)
            installed_exports.append(langserve_export)'''
serialized_extract_span = serialize(extract_span)

start, end = serialized_contents.find(serialized_extract_span), serialized_contents.find(serialized_extract_span) + len(serialized_extract_span)
print(start, end)

try:
    extractor = ExtractMethod(myproject, myresource, start, end)
    change_set = extractor.get_changes("helper")
    for change in change_set.changes:
        if change.old_contents is not None:
            change.old_contents = deserialize(change.old_contents)
        else:
            change.old_contents = deserialize(change.resource.read())
        change.new_contents = deserialize(change.new_contents)
    for change in change_set.changes:
        print(change.get_description())
except Exception as e:
    print(e)
finally:
    myresource.write(contents)

308 1693
--- a/test2.py
+++ b/test2.py
@@ -8,6 +8,9 @@
     installed_exports = []
     REPO_DIR = ""
     grouped = {}
+    helper(grouped, typer, update_repo, REPO_DIR, get_langserve_export, package_dir, copy_repo, installed_destination_paths, installed_exports)
+
+def helper(grouped, typer, update_repo, REPO_DIR, get_langserve_export, package_dir, copy_repo, installed_destination_paths, installed_exports):
     for (git, ref), group_deps in grouped.items():
         if len(group_deps) == 1:
             typer.echo(f"Adding {git}@{ref}...")



In [37]:
def extract_method(
    snippet,
    file_path,
    method_name,
    project_name="../../sweepai"
):
    project = rope.base.project.Project(project_name)

    resource = project.get_resource(file_path)
    contents = resource.read()
    start, end = contents.find(snippet), contents.find(snippet) + len(snippet)

    extractor = ExtractMethod(project, resource, start, end)
    change_set = extractor.get_changes(method_name, similar=True, global_=True)
    for change in change_set.changes:
        change.do()

    result = resource.read()

    for change in change_set.changes:
        change.undo()
    return result

snippet = r"""edit_sweep_comment(
                review_message + "\n\nSuccess! 🚀",
                4,
                pr_message=(
                    f"## Here's the PR! [{pr.html_url}]({pr.html_url}).\n{center(payment_message_start)}"
                ),
                done=True,
            )"""
print(extract_method(snippet, "handlers/on_ticket.py", "hydrate_sandbox"))

SyntaxError: f-string: invalid syntax (<string>, line 5)